To establish communication between Visual Odometry and Kalman filter, homogeneous transformation matrix is needed.
Currently, filters that adopt velocity motion model contains position and angle in x and z-axis to estimate the motion of an agent in a system.

To compute homogeneous transformation matrix, translation vector that contains position in x, y, and z coordinates, and rotation matrix, which can be derived from quaternion, are needed.
In this notebook, mathematical equation that explains new kinematic equation, which conforms to the velocity motion model, is derived.

When converting euler angles from quaternion, gimbal lock, which occurs when pitch angle (angle in y-axis) is +90 or -90 degree, needs to be taken into account.

In [2]:
import numpy as np
import sympy
from sympy.abc import alpha, x, y, v, w, R, psi, theta, phi
from sympy import symbols, Matrix

In [3]:
# state vector
px = symbols('p_{x}')
py = symbols('p_{y}')
pz = symbols('p_{z}')
qw = symbols('q_{w}')
qx = symbols('q_{x}')
qy = symbols('q_{y}')
qz = symbols('q_{z}')

# control input
v = symbols('v')
wx = symbols('w_{x}')
wy = symbols('w_{y}')
wz = symbols('w_{z}')

# delta time
dt = symbols('dt')

In [4]:
norm_w = symbols('|w|')
R = Matrix([
    [qw**2 + qx**2 - qy**2 - qz**2, 2*(qx*qy - qw*qz), 2*(qw*qy + qx*qz)],
    [2*(qx*qy + qw*qz), qw**2 - qx**2 + qy**2 - qz**2, 2*(qy*qz - qw*qx)],
    [2*(qx*qz - qw*qy), 2*(qw*qx + qy*qz), qw**2 - qx**2 - qy**2 + qz**2]
])
Omega = Matrix([
    [0, wz, -wy, wx],
    [-wz, 0, wx, wy],
    [wy, -wx, 0, wz],
    [-wx, -wy, -wz, 0]
])

## Normal condition

In [17]:
phi = sympy.atan2(2*(qw*qx + qy*qz), qw**2 - qx**2 - qy**2 + qz**2)
theta = sympy.asin(2*(qw*qy - qx*qz))
psi = sympy.atan2(2*(qw*qz + qx*qy), qw**2 + qx**2 - qy**2 - qz**2)

In [18]:
A = sympy.cos(norm_w*dt/2) * sympy.eye(4)
B = (1/norm_w)*sympy.sin(norm_w*dt/2)
q_t = Matrix([
    [qw],
    [qx],
    [qy],
    [qz]
])
(A + B * Omega) * q_t

Matrix([
[ q_{w}*cos(dt*|w|/2) + q_{x}*w_{z}*sin(dt*|w|/2)/|w| - q_{y}*w_{y}*sin(dt*|w|/2)/|w| + q_{z}*w_{x}*sin(dt*|w|/2)/|w|],
[-q_{w}*w_{z}*sin(dt*|w|/2)/|w| + q_{x}*cos(dt*|w|/2) + q_{y}*w_{x}*sin(dt*|w|/2)/|w| + q_{z}*w_{y}*sin(dt*|w|/2)/|w|],
[ q_{w}*w_{y}*sin(dt*|w|/2)/|w| - q_{x}*w_{x}*sin(dt*|w|/2)/|w| + q_{y}*cos(dt*|w|/2) + q_{z}*w_{z}*sin(dt*|w|/2)/|w|],
[-q_{w}*w_{x}*sin(dt*|w|/2)/|w| - q_{x}*w_{y}*sin(dt*|w|/2)/|w| - q_{y}*w_{z}*sin(dt*|w|/2)/|w| + q_{z}*cos(dt*|w|/2)]])

In [19]:
fxu = Matrix([
    [px - v*sympy.sin(psi)/wz + v*sympy.sin(psi + wz*dt)/wz],
    [py + v*sympy.cos(psi)/wz - v*sympy.cos(psi + wz*dt)/wz],
    [pz - v*sympy.sin(phi)/wx + v*sympy.sin(phi + wx*dt)/wx],
    [(A + B * Omega) * q_t]
])
fxu

Matrix([
[            p_{x} - v*(2*q_{w}*q_{z} + 2*q_{x}*q_{y})/(w_{z}*sqrt((2*q_{w}*q_{z} + 2*q_{x}*q_{y})**2 + (q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2)**2)) + v*sin(dt*w_{z} + atan2(2*q_{w}*q_{z} + 2*q_{x}*q_{y}, q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2))/w_{z}],
[p_{y} - v*cos(dt*w_{z} + atan2(2*q_{w}*q_{z} + 2*q_{x}*q_{y}, q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2))/w_{z} + v*(q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2)/(w_{z}*sqrt((2*q_{w}*q_{z} + 2*q_{x}*q_{y})**2 + (q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2)**2))],
[            p_{z} - v*(2*q_{w}*q_{x} + 2*q_{y}*q_{z})/(w_{x}*sqrt((2*q_{w}*q_{x} + 2*q_{y}*q_{z})**2 + (q_{w}**2 - q_{x}**2 - q_{y}**2 + q_{z}**2)**2)) + v*sin(dt*w_{x} + atan2(2*q_{w}*q_{x} + 2*q_{y}*q_{z}, q_{w}**2 - q_{x}**2 - q_{y}**2 + q_{z}**2))/w_{x}],
[                                                                                                                                              q_{w}*cos(dt*|w|/2) + q_{x}*w_{z}*sin(dt*|w|/2)/|w| - q_{y}*w_{y}

In [20]:
state_x = Matrix([px, py, pz, qw, qx, qy, qz])
control_input = Matrix([v, wx, wy, wz])

In [21]:
F = fxu.jacobian(state_x)
F

Matrix([
[1, 0, 0,            -2*q_{z}*v/(w_{z}*sqrt((2*q_{w}*q_{z} + 2*q_{x}*q_{y})**2 + (q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2)**2)) - v*(2*q_{w}*q_{z} + 2*q_{x}*q_{y})*(-2*q_{w}*(q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2) - 2*q_{z}*(2*q_{w}*q_{z} + 2*q_{x}*q_{y}))/(w_{z}*((2*q_{w}*q_{z} + 2*q_{x}*q_{y})**2 + (q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2)**2)**(3/2)) + v*(2*q_{w}*(-2*q_{w}*q_{z} - 2*q_{x}*q_{y})/((2*q_{w}*q_{z} + 2*q_{x}*q_{y})**2 + (q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2)**2) + 2*q_{z}*(q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2)/((2*q_{w}*q_{z} + 2*q_{x}*q_{y})**2 + (q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2)**2))*cos(dt*w_{z} + atan2(2*q_{w}*q_{z} + 2*q_{x}*q_{y}, q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2))/w_{z},            -2*q_{y}*v/(w_{z}*sqrt((2*q_{w}*q_{z} + 2*q_{x}*q_{y})**2 + (q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2)**2)) - v*(2*q_{w}*q_{z} + 2*q_{x}*q_{y})*(-2*q_{x}*(q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2) - 2*q_{y}*(2*q_{w}*q_{z} + 2*q_{x}*q_{y})

In [22]:
G = fxu.jacobian(control_input)
G

Matrix([
[            -(2*q_{w}*q_{z} + 2*q_{x}*q_{y})/(w_{z}*sqrt((2*q_{w}*q_{z} + 2*q_{x}*q_{y})**2 + (q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2)**2)) + sin(dt*w_{z} + atan2(2*q_{w}*q_{z} + 2*q_{x}*q_{y}, q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2))/w_{z},                                                                                                                                                                                                                                                                                                                                                                0,                        0,             dt*v*cos(dt*w_{z} + atan2(2*q_{w}*q_{z} + 2*q_{x}*q_{y}, q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2))/w_{z} + v*(2*q_{w}*q_{z} + 2*q_{x}*q_{y})/(w_{z}**2*sqrt((2*q_{w}*q_{z} + 2*q_{x}*q_{y})**2 + (q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2)**2)) - v*sin(dt*w_{z} + atan2(2*q_{w}*q_{z} + 2*q_{x}*q_{y}, q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2))/w_{z}*

In [24]:
values = {
    px: 0,
    py: 0,
    pz: 0,
    qw: 1,
    qx: 0,
    qy: 0,
    qz: 0,
    v: 1,
    wx: 1,
    wy: 1,
    wz: 1,
    dt: 0.1,
    norm_w: np.sqrt(1+1+1)
}

x_ = np.array(fxu.evalf(subs=values)).astype(float)
x_

array([[ 0.09983342],
       [ 0.00499583],
       [ 0.09983342],
       [ 0.99625234],
       [-0.04993752],
       [ 0.04993752],
       [-0.04993752]])

## when pitch = + pi/2 radian

https://danceswithcode.net/engineeringnotes/quaternions/quaternions.html

In [26]:
phi = 0
theta = sympy.asin(2*(qw*qy - qx*qz))
psi = -2*sympy.atan2(qx, qw)

A = sympy.cos(norm_w*dt/2) * sympy.eye(4)
B = (1/norm_w)*sympy.sin(norm_w*dt/2)
q_t = Matrix([
    [qw],
    [qx],
    [qy],
    [qz]
])
fxu = Matrix([
    [px - v*sympy.sin(psi)/wz + v*sympy.sin(psi + wz*dt)/wz],
    [py + v*sympy.cos(psi)/wz - v*sympy.cos(psi + wz*dt)/wz],
    [pz - v*sympy.sin(phi)/wx + v*sympy.sin(phi + wx*dt)/wx],
    [(A + B * Omega) * q_t]
])
F = fxu.jacobian(state_x)
G = fxu.jacobian(control_input)

In [27]:
fxu

Matrix([
[                          p_{x} + v*sin(dt*w_{z} - 2*atan2(q_{x}, q_{w}))/w_{z} + v*sin(2*atan2(q_{x}, q_{w}))/w_{z}],
[                          p_{y} - v*cos(dt*w_{z} - 2*atan2(q_{x}, q_{w}))/w_{z} + v*cos(2*atan2(q_{x}, q_{w}))/w_{z}],
[                                                                                       p_{z} + v*sin(dt*w_{x})/w_{x}],
[ q_{w}*cos(dt*|w|/2) + q_{x}*w_{z}*sin(dt*|w|/2)/|w| - q_{y}*w_{y}*sin(dt*|w|/2)/|w| + q_{z}*w_{x}*sin(dt*|w|/2)/|w|],
[-q_{w}*w_{z}*sin(dt*|w|/2)/|w| + q_{x}*cos(dt*|w|/2) + q_{y}*w_{x}*sin(dt*|w|/2)/|w| + q_{z}*w_{y}*sin(dt*|w|/2)/|w|],
[ q_{w}*w_{y}*sin(dt*|w|/2)/|w| - q_{x}*w_{x}*sin(dt*|w|/2)/|w| + q_{y}*cos(dt*|w|/2) + q_{z}*w_{z}*sin(dt*|w|/2)/|w|],
[-q_{w}*w_{x}*sin(dt*|w|/2)/|w| - q_{x}*w_{y}*sin(dt*|w|/2)/|w| - q_{y}*w_{z}*sin(dt*|w|/2)/|w| + q_{z}*cos(dt*|w|/2)]])

In [28]:
F

Matrix([
[1, 0, 0, 2*q_{x}*v*cos(dt*w_{z} - 2*atan2(q_{x}, q_{w}))/(w_{z}*(q_{w}**2 + q_{x}**2)) - 2*q_{x}*v*cos(2*atan2(q_{x}, q_{w}))/(w_{z}*(q_{w}**2 + q_{x}**2)), -2*q_{w}*v*cos(dt*w_{z} - 2*atan2(q_{x}, q_{w}))/(w_{z}*(q_{w}**2 + q_{x}**2)) + 2*q_{w}*v*cos(2*atan2(q_{x}, q_{w}))/(w_{z}*(q_{w}**2 + q_{x}**2)),                        0,                       0],
[0, 1, 0, 2*q_{x}*v*sin(dt*w_{z} - 2*atan2(q_{x}, q_{w}))/(w_{z}*(q_{w}**2 + q_{x}**2)) + 2*q_{x}*v*sin(2*atan2(q_{x}, q_{w}))/(w_{z}*(q_{w}**2 + q_{x}**2)), -2*q_{w}*v*sin(dt*w_{z} - 2*atan2(q_{x}, q_{w}))/(w_{z}*(q_{w}**2 + q_{x}**2)) - 2*q_{w}*v*sin(2*atan2(q_{x}, q_{w}))/(w_{z}*(q_{w}**2 + q_{x}**2)),                        0,                       0],
[0, 0, 1,                                                                                                                                                  0,                                                                                                                    

In [29]:
G

Matrix([
[ sin(dt*w_{z} - 2*atan2(q_{x}, q_{w}))/w_{z} + sin(2*atan2(q_{x}, q_{w}))/w_{z},                                                   0,                        0, dt*v*cos(dt*w_{z} - 2*atan2(q_{x}, q_{w}))/w_{z} - v*sin(dt*w_{z} - 2*atan2(q_{x}, q_{w}))/w_{z}**2 - v*sin(2*atan2(q_{x}, q_{w}))/w_{z}**2],
[-cos(dt*w_{z} - 2*atan2(q_{x}, q_{w}))/w_{z} + cos(2*atan2(q_{x}, q_{w}))/w_{z},                                                   0,                        0, dt*v*sin(dt*w_{z} - 2*atan2(q_{x}, q_{w}))/w_{z} + v*cos(dt*w_{z} - 2*atan2(q_{x}, q_{w}))/w_{z}**2 - v*cos(2*atan2(q_{x}, q_{w}))/w_{z}**2],
[                                                            sin(dt*w_{x})/w_{x}, dt*v*cos(dt*w_{x})/w_{x} - v*sin(dt*w_{x})/w_{x}**2,                        0,                                                                                                                                           0],
[                                                                              0, 

## When pitch = - pi/2 radians

https://danceswithcode.net/engineeringnotes/quaternions/quaternions.html

In [30]:
phi = 0
theta = sympy.asin(2*(qw*qy - qx*qz))
psi = 2*sympy.atan2(qx, qw)
A = sympy.cos(norm_w*dt/2) * sympy.eye(4)
B = (1/norm_w)*sympy.sin(norm_w*dt/2)
q_t = Matrix([
    [qw],
    [qx],
    [qy],
    [qz]
])
fxu = Matrix([
    [px - v*sympy.sin(psi)/wz + v*sympy.sin(psi + wz*dt)/wz],
    [py + v*sympy.cos(psi)/wz - v*sympy.cos(psi + wz*dt)/wz],
    [pz - v*sympy.sin(phi)/wx + v*sympy.sin(phi + wx*dt)/wx],
    [(A + B * Omega) * q_t]
])
F = fxu.jacobian(state_x)
G = fxu.jacobian(control_input)

In [31]:
fxu

Matrix([
[                          p_{x} + v*sin(dt*w_{z} + 2*atan2(q_{x}, q_{w}))/w_{z} - v*sin(2*atan2(q_{x}, q_{w}))/w_{z}],
[                          p_{y} - v*cos(dt*w_{z} + 2*atan2(q_{x}, q_{w}))/w_{z} + v*cos(2*atan2(q_{x}, q_{w}))/w_{z}],
[                                                                                       p_{z} + v*sin(dt*w_{x})/w_{x}],
[ q_{w}*cos(dt*|w|/2) + q_{x}*w_{z}*sin(dt*|w|/2)/|w| - q_{y}*w_{y}*sin(dt*|w|/2)/|w| + q_{z}*w_{x}*sin(dt*|w|/2)/|w|],
[-q_{w}*w_{z}*sin(dt*|w|/2)/|w| + q_{x}*cos(dt*|w|/2) + q_{y}*w_{x}*sin(dt*|w|/2)/|w| + q_{z}*w_{y}*sin(dt*|w|/2)/|w|],
[ q_{w}*w_{y}*sin(dt*|w|/2)/|w| - q_{x}*w_{x}*sin(dt*|w|/2)/|w| + q_{y}*cos(dt*|w|/2) + q_{z}*w_{z}*sin(dt*|w|/2)/|w|],
[-q_{w}*w_{x}*sin(dt*|w|/2)/|w| - q_{x}*w_{y}*sin(dt*|w|/2)/|w| - q_{y}*w_{z}*sin(dt*|w|/2)/|w| + q_{z}*cos(dt*|w|/2)]])

In [32]:
F

Matrix([
[1, 0, 0, -2*q_{x}*v*cos(dt*w_{z} + 2*atan2(q_{x}, q_{w}))/(w_{z}*(q_{w}**2 + q_{x}**2)) + 2*q_{x}*v*cos(2*atan2(q_{x}, q_{w}))/(w_{z}*(q_{w}**2 + q_{x}**2)), 2*q_{w}*v*cos(dt*w_{z} + 2*atan2(q_{x}, q_{w}))/(w_{z}*(q_{w}**2 + q_{x}**2)) - 2*q_{w}*v*cos(2*atan2(q_{x}, q_{w}))/(w_{z}*(q_{w}**2 + q_{x}**2)),                        0,                       0],
[0, 1, 0, -2*q_{x}*v*sin(dt*w_{z} + 2*atan2(q_{x}, q_{w}))/(w_{z}*(q_{w}**2 + q_{x}**2)) + 2*q_{x}*v*sin(2*atan2(q_{x}, q_{w}))/(w_{z}*(q_{w}**2 + q_{x}**2)), 2*q_{w}*v*sin(dt*w_{z} + 2*atan2(q_{x}, q_{w}))/(w_{z}*(q_{w}**2 + q_{x}**2)) - 2*q_{w}*v*sin(2*atan2(q_{x}, q_{w}))/(w_{z}*(q_{w}**2 + q_{x}**2)),                        0,                       0],
[0, 0, 1,                                                                                                                                                   0,                                                                                                                   

In [33]:
G

Matrix([
[ sin(dt*w_{z} + 2*atan2(q_{x}, q_{w}))/w_{z} - sin(2*atan2(q_{x}, q_{w}))/w_{z},                                                   0,                        0, dt*v*cos(dt*w_{z} + 2*atan2(q_{x}, q_{w}))/w_{z} - v*sin(dt*w_{z} + 2*atan2(q_{x}, q_{w}))/w_{z}**2 + v*sin(2*atan2(q_{x}, q_{w}))/w_{z}**2],
[-cos(dt*w_{z} + 2*atan2(q_{x}, q_{w}))/w_{z} + cos(2*atan2(q_{x}, q_{w}))/w_{z},                                                   0,                        0, dt*v*sin(dt*w_{z} + 2*atan2(q_{x}, q_{w}))/w_{z} + v*cos(dt*w_{z} + 2*atan2(q_{x}, q_{w}))/w_{z}**2 - v*cos(2*atan2(q_{x}, q_{w}))/w_{z}**2],
[                                                            sin(dt*w_{x})/w_{x}, dt*v*cos(dt*w_{x})/w_{x} - v*sin(dt*w_{x})/w_{x}**2,                        0,                                                                                                                                           0],
[                                                                              0, 

In [37]:
np.eye(6).shape

(6, 6)